Step 0: Install Required Libraries

In [ ]:
%pip install tensorflow==2.16 keras==3.8 matplotlib seaborn scikit-learn notebook pandas

Step 1: Import libraries

In [ ]:
import tensorflow as tf
from keras.api.models import Sequential
from keras.api.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.api.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.api.preprocessing import image_dataset_from_directory
from keras.api.applications import ResNet50
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

Cell 2: Define Dataset Paths and Parameters

In [ ]:
# Set the paths to your dataset
TRAINING_DIR = '/Users/dianasill/Documents/Projects/PneumoniaXRayML/pneumonia_detection_model/chest_xray/train'
VALIDATION_DIR = '/Users/dianasill/Documents/Projects/PneumoniaXRayML/pneumonia_detection_model/chest_xray/val'
TEST_DIR = '/Users/dianasill/Documents/Projects/PneumoniaXRayML/pneumonia_detection_model/chest_xray/test'

# Parameters
IMG_SIZE = 150  # Resize the images to this size
BATCH_SIZE = 32
EPOCHS = 15

Cell 3: Load and Preprocess Datasets

In [ ]:
# Load and preprocess the dataset using image_dataset_from_directory
train_dataset = image_dataset_from_directory(
    TRAINING_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary',
    validation_split=0.2,
    subset="training",
    seed=123
)

val_dataset = image_dataset_from_directory(
    VALIDATION_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary',
    validation_split=0.2,
    subset="validation",
    seed=123
)

test_dataset = image_dataset_from_directory(
    TEST_DIR,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary',
    seed=123
)

# Configure datasets for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

Cell 4: Define Data Augmentation Layer

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomBrightness(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

Cell 5: Build and Compile the Model

In [ ]:
# Transfer learning with ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model layers initially
base_model.trainable = False

# Model Architecture
model = Sequential([
    data_augmentation,  # Data Augmentation Layer
    base_model,  # Pretrained ResNet50
    GlobalAveragePooling2D(),  # Global Average Pooling to reduce dimensions
    Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Dense layer with L2 regularization
    Dropout(0.5),  # Dropout for regularization
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

Cell 6: Train the Model

In [ ]:
# Early stopping and ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# Train the model
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=[early_stopping, lr_scheduler]
)

Cell 7: Fine-Tune the Model

In [ ]:
# Fine-tune the model: Unfreeze some layers
base_model.trainable = True
for layer in base_model.layers[:143]:
    layer.trainable = False

# Recompile the model for fine-tuning
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              metrics=['accuracy'])

# Train the model further
history_finetune = model.fit(
    train_dataset,
    # change EPOCHS to something more manageable
    epochs=7,
    validation_data=val_dataset,
    callbacks=[early_stopping, lr_scheduler]
)

# Save the model
model.save('pneumonia_detection_model_finetuned.keras')

Cell 8: Plot Training Results

In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_history(history_finetune)

Cell 9: Evaluate and Visualize Results

In [ ]:
def predict_all_images(dataset):
    true_labels = []
    pred_labels = []

    for images, labels in dataset:
        preds = model.predict(images)
        true_labels.extend(labels.numpy())
        pred_labels.extend((preds > 0.5).astype(int))

    return true_labels, pred_labels

def plot_confusion_matrix(true_labels, pred_labels):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# Validation Set Results
true_labels, pred_labels = predict_all_images(val_dataset)
print("Validation Set Results:")
print(classification_report(true_labels, pred_labels, target_names=['Normal', 'Pneumonia']))
plot_confusion_matrix(true_labels, pred_labels)

# Test Set Results
true_labels_test, pred_labels_test = predict_all_images(test_dataset)
print("Test Set Results:")
print(classification_report(true_labels_test, pred_labels_test, target_names=['Normal', 'Pneumonia']))
plot_confusion_matrix(true_labels_test, pred_labels_test)